In [1]:
import sys
sys.path.append('./stp-gsr')  # Add the directory to Python path

from src.models.stp_gsr import STPGSR
from src.dual_graph_utils import revert_dual
from src.dataset import load_dataset

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from evaluation import evaluate_matrices
from MatrixVectorizer import MatrixVectorizer
from mlp_baseline import NaiveMLP

import yaml
from omegaconf import OmegaConf

config = {
    'defaults': ['_self_', 'dataset: custom', 'model: stp_gsr'],
    
    'dataset': {
        'name': 'custom',
        'n_samples': 167,
        'n_source_nodes': 160,
        'n_target_nodes': 268,
        'node_feat_init': 'adj',
        'node_feat_dim': 160  # same as n_source_nodes
    },
    
    'model': {
        'name': 'stp_gsr',
        'target_edge_initializer': {
            'num_heads': 4,
            'edge_dim': 1,
            'dropout': 0.2,
            'beta': False
        },
        'dual_learner': {
            'in_dim': 1,
            'out_dim': 1,
            'num_heads': 1,
            'dropout': 0.2,
            'beta': False
        }
    },
    
    'experiment': {
        'n_epochs': 60,
        'batch_size': 16,
        'lr': 0.001,
        'log_val_loss': False,
        'base_dir': 'results',
        'run_name': 'run1',
        'kfold': {
            'n_splits': 3,
            'shuffle': True,
            'random_state': 42
        }
    }
}

config = OmegaConf.create(config)

In [3]:
# Load the test data and the model, and evaluate the model on the test data.
source_data, target_data = load_dataset(config)
MODEL_FILE_NAME = "stp-gsr/results/stp_gsr_residual_2layers_preactivation_no_graph_conv_whole_data/custom/run1/fold_1/model.pth"

model = STPGSR(config)
model.load_state_dict(torch.load(MODEL_FILE_NAME))
model.eval()

STPGSR(
  (target_edge_initializer): TargetEdgeInitializer(
    (residual_proj): Linear(in_features=160, out_features=268, bias=True)
    (convs): ModuleList(
      (0): TransformerConv(160, 16, heads=4)
      (1): TransformerConv(64, 67, heads=4)
    )
    (bns): ModuleList(
      (0): GraphNorm(160)
      (1): GraphNorm(64)
    )
  )
  (dual_learner): DualGraphLearner(
    (conv1): TransformerConv(1, 1, heads=1)
    (bn1): GraphNorm(1)
    (out_proj): Linear(in_features=1, out_features=1, bias=True)
  )
)

In [3]:
# Change test data to torch tensor
n_target_nodes = config.dataset.n_target_nodes  # n_t
    
eval_output = []
with torch.no_grad():
    for source, target in zip(source_data, target_data):
        source_g = source['pyg']    
        target_m = target['mat']    # (n_t, n_t)
        print(target_m)
        model_pred, model_target = model(source_g, target_m) 

        pred_m = revert_dual(model_pred, n_target_nodes)    # (n_t, n_t)
        pred_m = pred_m.cpu().numpy()


        eval_output.append(pred_m)




return eval_output, eval_loss

# Predict the test data
n_target_nodes = config.dataset.n_target_nodes  # n_t
model_pred = model(test_data)
y_pred = revert_dual(model_pred, n_target_nodes)    # (n_t, n_t)
y_pred = y_pred.cpu().numpy()

print(y_pred.shape)

TypeError: STPGSR.forward() missing 1 required positional argument: 'target_mat'

In [12]:
# flatten
y_pred = y_pred.detach().numpy()
y_pred = y_pred.flatten()
print(y_pred.shape)

(4007136,)


In [13]:
# Convert tensor to csv
df = pd.DataFrame({
    'ID': np.arange(1, len(y_pred)+1),
    'Predicted': y_pred
})

df.to_csv('prediction.csv', index=False)
print('Prediction saved to prediction.csv')

Prediction saved to prediction.csv
